<a href="https://colab.research.google.com/github/Kanyinsola-O/CE706/blob/main/IR3/ElasticsearchCORD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Importing correct modules for task

In [103]:
#Step 1 Import modules 

!pip install elasticsearch

try:   
    import os
    import sys
    import csv
    
    import elasticsearch
    from elasticsearch import Elasticsearch 
    import pandas as pd
    import numpy as np
    from elasticsearch import helpers
    
    print("All Modules Loaded !")
except Exception as e:
    print("Some Modules are Missing {}".format(e))   

All Modules Loaded !


##Securing relevant files 

In [104]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Connecting Elasticsearch to Jupyter

In [105]:
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.6.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.6.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.6.2
### END OF BLOCK ###

import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.6.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 40
!curl -X GET "localhost:9200/"

{
  "name" : "56f4a0fbd750",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "AkXNMrQ5SZiWKIHMju1zzQ",
  "version" : {
    "number" : "7.0.0",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "b7e28a7",
    "build_date" : "2019-04-05T22:55:32.697037Z",
    "build_snapshot" : false,
    "lucene_version" : "8.0.0",
    "minimum_wire_compatibility_version" : "6.7.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [106]:
#Step Connect to ES

def connect_elasticsearch():
    es = None
    es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
    if es.ping():
        print('Yupiee  Connected ')
    else:
        print('Awww it could not connect!')
    return es
es = connect_elasticsearch()

Yupiee  Connected 


##Obtaining and read the dataset

In [107]:
df = pd.read_csv('metadata.csv', low_memory=False)
print(df.shape)

(450385, 19)


In [108]:
df.columns

Index(['cord_uid', 'sha', 'source_x', 'title', 'doi', 'pmcid', 'pubmed_id',
       'license', 'abstract', 'publish_time', 'authors', 'journal', 'mag_id',
       'who_covidence_id', 'arxiv_id', 'pdf_json_files', 'pmc_json_files',
       'url', 's2_id'],
      dtype='object')

In [109]:
df.shape

(450385, 19)

In [110]:
df["cord_uid"]. nunique()

428883

##Creating Elasticsearch instance

In [111]:
ENDPOINT = "http://localhost:9200/"
es = Elasticsearch(timeout=600, hosts=ENDPOINT)

##Preprocessing data before ES upload

In [112]:
#df.isna().sum()

In [113]:
#df= df.dropna()

In [114]:
df.shape

(450385, 19)

In [115]:
df2 = df.to_dict('records')

##Conversion of data to one appropiate for ES

##ELK Formatting

In [116]:
df.head(2)

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN


In [117]:
df.columns

Index(['cord_uid', 'sha', 'source_x', 'title', 'doi', 'pmcid', 'pubmed_id',
       'license', 'abstract', 'publish_time', 'authors', 'journal', 'mag_id',
       'who_covidence_id', 'arxiv_id', 'pdf_json_files', 'pmc_json_files',
       'url', 's2_id'],
      dtype='object')

In [118]:
def generator(df2):
    for c, line in enumerate(df2):
        yield {
    '_index': 'cord',
    '_type': '_doc',
    '_id': c,
    '_source': {
        'cord_uid':line.get("cord_uid", ""),
      
    }
        }
    raise StopIteration

In [119]:
df2 = df.to_dict('records')

In [120]:
mygen = generator(df2)

In [121]:
mygen

<generator object generator at 0x7f67b7dde8d0>

In [122]:
next(mygen)

{'_id': 0,
 '_index': 'cord',
 '_source': {'cord_uid': 'ug7v899j'},
 '_type': '_doc'}

#

In [124]:
es.ping()

True

In [125]:
#try:
    res = helpers.bulk(es, generator(df2))
    print("Working")
except Exception as e:
    pass

/usr/local/lib/python3.7/dist-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


In [126]:
#df.isna().sum()

##Creating an Index 

In [131]:
#es.indices.create(index='test-index', ignore=400)
#es.indices.create(index='test-index1', ignore=400)

In [130]:
try:
    import elasticsearch

    from elasticsearch import Elasticsearch
    from elasticsearch import helpers

    import pandas as pd
    import json

    import datetime
    import os
    import sys

    import os
    import numpy as np

    print("Loaded  .. . . . . . . .")
except Exception as E:
    print("Some Modules are Missing {} ".format(e))

class Reader(object):

    def __init__(self,filename,size, indexName='cords'):
        self.filename=filename
        self.size=size
        self.indexName =indexName

    def process(self):
        df = pd.read_csv(self.filename, chunksize=self.size)
        df = next(df)
        df2 = df.to_dict("records")
        df2 = self.removeNan(df2=df2)
        return df2

    def create_columns(self, record, column_name):
        value = record.get(column_name, "").__str__()
        if len(column_name) == 0 or column_name == "": value = ""
        if 'nan' in value: value = ""

        return value

    def removeNan(self, df2):
        records = []
        for c, x in enumerate(df2):
            jsonRecord = {}
            jsonRecord['cord_uid']          = self.create_columns(record=x, column_name='cord_uid')
            jsonRecord['sha']               = self.create_columns(record=x, column_name='sha')
            jsonRecord['source_x']          =self.create_columns(record=x, column_name='source_x')
            jsonRecord['title']             =self.create_columns(record=x, column_name='title')
            jsonRecord['doi']               =self.create_columns(record=x, column_name='doi')
            jsonRecord['pmcid']             =self.create_columns(record=x, column_name='pmcid')
            jsonRecord['pubmed_id']         =self.create_columns(record=x, column_name='pubmed_id')
            jsonRecord['license']           =self.create_columns(record=x, column_name='license')
            jsonRecord['abstract']          =self.create_columns(record=x, column_name='abstract')
            jsonRecord['publish_time']      =self.create_columns(record=x, column_name='publish_time')
            jsonRecord['authors']           =self.create_columns(record=x, column_name='authors')
            jsonRecord['journal']           =self.create_columns(record=x, column_name='journal')
            jsonRecord['mag_id']            =self.create_columns(record=x, column_name='mag_id')
            jsonRecord['who_covidence_id']  =self.create_columns(record=x, column_name='who_covidence_id')
            jsonRecord['arxiv_id']          =self.create_columns(record=x, column_name='arxiv_id')
            jsonRecord['pdf_json_files']    =self.create_columns(record=x, column_name='pdf_json_files')
            jsonRecord['pmc_json_files']    =self.create_columns(record=x, column_name='pmc_json_files')
            jsonRecord['url']               =self.create_columns(record=x, column_name='url')
            jsonRecord['s2_id']             =self.create_columns(record=x, column_name='s2_id')
            json_elk_format = {
                '_index': self.indexName,
                '_type': '_doc',
                '_id': c,
                '_source': jsonRecord
            }
            records.append(json_elk_format)
        return records


def main():

    _helper = Reader(filename='metadata.csv', size=1000, indexName='cords')
    _data = _helper.process()

    ENDPOINT = "http://localhost:9200/"
    es = Elasticsearch(timeout=600,hosts=ENDPOINT)

    try:
        res = helpers.bulk(es, _data)
    except Exception as e:
        pass
    print('done')


main()



Loaded  .. . . . . . . .


/usr/local/lib/python3.7/dist-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


done
